In [1]:
from alntk.alignment import import_from_fasta, get_unaligned_seqs, get_compact_alignment, write_to_fasta
from alntk.plotting import default_plot_style

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import coo_array
import matplotlib as mpl
import matplotlib.pyplot as plt

In [3]:
color_cycle = default_plot_style()
data_folder = '../data/'

# Compare the choices in iterative alignment

As Olivier suggested, we can compare:

- the alignment with all the sequences and all the positions
- the alignment with all the sequences and the positions eliminated by the iterative process

Let's see what kind of frequency vector and correlation matrix do these yield.

In [4]:
new_aln_descs, new_aln_seqs = import_from_fasta(data_folder + 'new_aln.faa')

In [5]:
subaln_size = 10_000

In [6]:
np.random.seed(42)
subsample_mask = np.random.randint(0, len(new_aln_descs), size=subaln_size)
subaln_seqs = new_aln_seqs[subsample_mask]
subaln_descs = new_aln_descs[subsample_mask]

In [7]:
write_to_fasta(subaln_descs, subaln_seqs, data_folder + 'new_aln_subsample.faa')

In [20]:
aas = np.array(list('-ACDEFGHIKLMNPQRSTVWY'))
len_aas = len(aas)
aas_to_num_dict = {}
for k, v in dict(enumerate(aas)).items():
    aas_to_num_dict[v] = k

In [21]:
def aas_to_num(a, aas_to_num_dict=aas_to_num_dict):
    return np.vectorize(aas_to_num_dict.__getitem__)(a)

In [45]:
subaln_num = aas_to_num(subaln_seqs)

In [46]:
subaln_num

array([[ 8, 18,  6, ...,  0,  0,  0],
       [ 8, 18,  6, ...,  0,  0,  0],
       [ 8, 18,  6, ...,  0,  0,  0],
       ...,
       [18, 18, 12, ...,  0,  0,  0],
       [18, 18,  6, ...,  0,  0,  0],
       [ 8, 18, 16, ...,  0,  0,  0]])

# SANDBOX

In [7]:
iterative_pos = np.array([0,1,2,3,35,36,37,38,39,40,82,83,84,85,86,87,88,89,90,91,92,93,94,110,111,112,113,114,115,116,117,118,119,120,121,122,129,130,131,132,133,134,147,148,149,150,151,152,153,154,155,156,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,280,281,282,283,307,308,309,310,311,312,313,333,334,335,336,337,338,352,353,354,355,356,357,358,359,360,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,399,400,401,421,422,423,424,425,430,431,433,434,435,436,437,438,439,440,441,469,470,471,472,473,474,475,476,477,478,494,495,496,497,498,499,500,501,502,503,504,505,506,507,508,509,510,520,521,522,523,524,525,526,527,528,530,531,532,533,534,535,536,537,538,544,545,546,547,548,549,550,551,552,553,554,555,556,557,563,564,565,566,567,568,569,575,576,577,578,579,580,581,582,583,584,591,592,593,594,595,596,597,598,599,600,601,602,603,604,605])

In [8]:
iter_subaln_seqs = subaln_seqs[:, iterative_pos]

In [9]:
subaln_ohe = pd.get_dummies(pd.DataFrame(subaln_seqs).T)

In [10]:
iter_subaln_ohe = pd.get_dummies(pd.DataFrame(iter_subaln_seqs).T)

We have two matrices. There is a column for each element of the cartesian product Positions x Amino acids.

In [12]:
freq_matrix = np.zeros((subaln_size, len_aas))

In [13]:
for k, v in subaln_ohe.sum().items():
    pos_idx = int(k.split('_')[0])
    aa_idx = np.where(k.split('_')[1] == aas)[0][0]
    freq_matrix[pos_idx, aa_idx] = v / 693.

In [14]:
corr_tensor = np.zeros((subaln_size, subaln_size, len_aas, len_aas))

In [15]:
for ki, vi in subaln_ohe.sum().items():
    for kj, vj in subaln_ohe.sum().items():
        pos_idx_i = int(ki.split('_')[0])
        aa_idx_i = np.where(ki.split('_')[1] == aas)[0][0]
        pos_idx_j = int(kj.split('_')[0])
        aa_idx_j = np.where(kj.split('_')[1] == aas)[0][0]
        corr_tensor[pos_idx_i, pos_idx_j, aa_idx_i, aa_idx_j] = v / (693. * 693.)

# Iter

In [16]:
iter_subaln_seqs.shape

(100, 260)

In [17]:
iter_freq_matrix = np.zeros((subaln_size, len_aas))

In [18]:
for k, v in iter_subaln_ohe.sum().items():
    pos_idx = int(k.split('_')[0])
    aa_idx = np.where(k.split('_')[1] == aas)[0][0]
    iter_freq_matrix[pos_idx, aa_idx] = v / 260.

In [19]:
iter_corr_tensor = np.zeros((subaln_size, subaln_size, len_aas, len_aas))

In [20]:
for ki, vi in iter_subaln_ohe.sum().items():
    for kj, vj in iter_subaln_ohe.sum().items():
        pos_idx_i = int(ki.split('_')[0])
        aa_idx_i = np.where(ki.split('_')[1] == aas)[0][0]
        pos_idx_j = int(kj.split('_')[0])
        aa_idx_j = np.where(kj.split('_')[1] == aas)[0][0]
        iter_corr_tensor[pos_idx_i, pos_idx_j, aa_idx_i, aa_idx_j] = v / (260. * 260.)